In [1]:
# .env 파일을 읽어서 환경변수로 설정
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
Runnable


## Langchain에서 데이터를 효과적으로 전달하는 방법
1. RunnablePassthrough
2. RunnableParallel
3. RunnableLambda


### 1. `RunnablePassthrough`: 데이터를 그대로 넘겨(통과시켜)주는 역할

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{word}를 영어로?")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = prompt | llm
chain

PromptTemplate(input_variables=['word'], input_types={}, partial_variables={}, template='{word}를 영어로?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fa50d4f4ee0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fa50d4f6fe0>, root_client=<openai.OpenAI object at 0x7fa524448e20>, root_async_client=<openai.AsyncOpenAI object at 0x7fa50d4f4f40>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

chain 을 `invoke()` 하여 실행할 때는 입력 데이터의 타입은 ***딕셔너리***

In [4]:
# chain 을 실행합니다.
chain.invoke({"word": '사과'})

AIMessage(content='apple', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bbbeed0e-6fe0-472b-b8d1-af0039e72bab-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

But, 1개의 변수만 템플릿에 포함하고 있다면, 값만 전달하는 것도 가능

In [5]:
# chain 을 실행합니다.
chain.invoke('사과')

AIMessage(content='apple', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fc3e3443-a372-4824-8512-07bb7481faf7-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [6]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough 는 runnable 객체이며, runnable 객체는 invoke() 메소드를 사용하여 별도 실행이 가능
# RunnablePassthrough()를 사용하여 체인 구성
runnable_chain = {"word": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경됨
runnable_chain.invoke('인공지능')

AIMessage(content='Artificial Intelligence', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 17, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c72479d6-67e9-47ac-8f66-200d762c3cb6-0', usage_metadata={'input_tokens': 17, 'output_tokens': 3, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### 2. `RunnableParallel`: 여러 작업을 동시에(병렬)로 처리하도록 도와주는 도구

In [8]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [9]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울특별시입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 19, 'total_tokens': 29, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9dd8f3a7-1b89-419f-955f-81d0f1603932-0', usage_metadata={'input_tokens': 19, 'output_tokens': 10, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 총 면적은 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 20, 'total_tokens': 43, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name

### 3. `RunnableLambda`: 사용자 정의 함수 매핑하도록 도와주는 도구


In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def concat_output(text):
    return text['capital'].content + ' ' + text['area'].content

In [11]:
final_chain = (combined_chain
            | {'info': RunnableLambda(concat_output)} 
            | PromptTemplate.from_template("{info}의 내용을 자연스럽게 다듬고, 이모지를 넣어줘.")
            | ChatOpenAI())

final_chain.invoke("대한민국")

AIMessage(content='안녕하세요! 서울특별시에 오신 것을 환영합니다! 🌆 대한민국의 총 면적은 약 100,363㎢로, 다채로운 자연경관과 역사적인 명소로 가득한 아름다운 나라입니다. 🇰🇷 함께 즐거운 시간 보내시길 바라요! 🌟', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 69, 'total_tokens': 185, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c8afdc06-723d-47f2-8774-b41038f9d05b-0', usage_metadata={'input_tokens': 69, 'output_tokens': 116, 'total_tokens': 185, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

'Oct-14'

In [15]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 대한민국 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
# 출력
print(chain.invoke(3))

다음은 10월 14일에 생일인 대한민국의 유명인 3명입니다:

1. **이정재** - 1973년 10월 14일
2. **김범수** - 1979년 10월 14일
3. **이상민** - 1973년 10월 14일

이 외에도 다른 유명인들이 있을 수 있지만, 위의 세 명이 대표적인 인물입니다.


### [실습] Runnnable 세가지를 적절히 사용하여 챗봇 생성

- 위 코드를 활용하여 아래 내용에 맞게 작성하시요.
- RunnableParallel 사용
    - chain1 : `{food}` 재료로 만들 수 있는 메인 디시를 묻는 체인
    - chain2 : `{food}` 재료로 만들 수 있는 후식 디저트를 묻는 체인
- RunnableLambda 사용
    - 두 체인의 결과 값 이어 붙이기 -> `{info}`
    - `{info}`를 바탕으로 이모지를 사용하여 우리 음식점 메뉴 추천 홍보문구를 작성해주는 템플릿


In [20]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# 첫 번째 체인: 재료로 만들 수 있는 메인 디시를 묻는 체인
chain1 = (
    {'food': RunnablePassthrough()}
    | PromptTemplate.from_template("{food}로 만들 수 있는 메인 디시는?")
    | ChatOpenAI(model_name='gpt-4o-mini')
)

# 두 번째 체인: 재료로 만들 수 있는 후식 디저트를 묻는 체인
chain2 = (
        {'food': RunnablePassthrough()}
    | PromptTemplate.from_template("{food}로 만들 수 있는 후식 디저트는?")
    | ChatOpenAI(model_name='gpt-4o-mini')
)

# 두 체인을 병렬로 실행
combined_chain =  RunnableParallel(main=chain1, dessert=chain2)

# 두 체인의 결과를 이어 붙이는 함수
def concat_food_info(text):
    
    return text['main'].content + ' ' + text['dessert'].content

# {info}를 바탕으로 이모지를 사용하여 우리 음식점 메뉴 추천 홍보문구를 작성해주는 템플릿
final_chain = ( 
               combined_chain
               |{'info':RunnableLambda(concat_food_info)}
               | PromptTemplate.from_template("{info}의 내용을 자연스럽게 다듬고, 레시피를 넣어줘")
               | ChatOpenAI(model_name = 'gpt-4o-mini')
)
# final_chain.invoke("감자")
# # prompt = 

# 최종 체인은 두 체인의 결과를 이어 붙이고 우리 가게 홍보 문구를 생성하는 작업

# 사용자가 생성할 메뉴의 개수를 입력받아 처리
final_output = final_chain.invoke({'food':'감자'})

# 결과 출력
print(final_output)


content='감자는 다양한 요리에 활용될 수 있는 만능 재료입니다. 아래는 감자를 활용한 여러 가지 메인 요리와 후식 디저트의 예시와 간단한 레시피를 소개합니다.\n\n### 메인 요리\n\n1. **감자전**\n   - **재료**: 감자 3개, 양파 1개, 부침가루 1컵, 소금 약간, 식용유\n   - **레시피**:\n     1. 감자와 양파를 곱게 갈아줍니다.\n     2. 갈아낸 감자와 양파에 부침가루와 소금을 넣고 잘 섞어줍니다.\n     3. 팬에 식용유를 두르고 반죽을 동그랗게 올려 중불에서 양면이 노릇할 때까지 부칩니다.\n\n2. **감자탕**\n   - **재료**: 감자 2개, 돼지등뼈 500g, 대파, 마늘, 고추장, 간장, 고춧가루\n   - **레시피**:\n     1. 돼지등뼈를 깨끗이 씻고, 냄비에 물과 함께 넣어 끓여 냄비의 거품을 제거합니다.\n     2. 감자와 대파, 마늘을 넣고, 고추장, 간장, 고춧가루로 간을 맞춥니다.\n     3. 모든 재료가 부드러워질 때까지 끓입니다.\n\n3. **감자조림**\n   - **재료**: 감자 4개, 간장 3큰술, 설탕 1큰술, 물 1컵, 참기름\n   - **레시피**:\n     1. 감자를 깍둑 썰어 냄비에 넣습니다.\n     2. 간장, 설탕, 물을 넣고 끓입니다.\n     3. 중불에서 감자가 익을 때까지 조리며, 마지막에 참기름을 넣어줍니다.\n\n4. **감자스프**\n   - **재료**: 감자 3개, 양파 1개, 마늘 2쪽, 생크림 1컵, 소금, 후추\n   - **레시피**:\n     1. 감자, 양파, 마늘을 잘게 썰어 냄비에 넣고 물을 부어 끓입니다.\n     2. 재료가 부드러워지면 블렌더로 갈아줍니다.\n     3. 생크림과 소금, 후추로 간을 맞춰 다시 끓입니다.\n\n5. **감자 샐러드**\n   - **재료**: 삶은 감자 3개, 마요네즈 1/2컵, 양파 1/4개, 피클 2개, 소금, 후추\n   - **레시피**:\n     

In [21]:
final_output = final_chain.stream({'food':'감자'})


In [22]:
from langchain_teddynote.messages import stream_response
final_output = final_chain.stream({'food':'감자'})
stream_response(final_output)

감자는 다양한 요리에서 활용될 수 있는 다재다능한 식재료입니다. 여기서는 감자로 만들 수 있는 몇 가지 메인 요리와 후식을 소개하고, 각 요리에 대한 간단한 레시피를 함께 제공하겠습니다.

### 메인 요리

1. **감자조림**
   - **재료**: 감자 3개, 간장 3큰술, 설탕 1큰술, 마늘 2쪽, 물 1컵, 참기름 약간
   - **레시피**:
     1. 감자를 껍질을 벗기고 먹기 좋은 크기로 자릅니다.
     2. 팬에 감자와 간장, 설탕, 마늘, 물을 넣고 끓입니다.
     3. 중불에서 감자가 부드러워질 때까지 15분 정도 조립니다.
     4. 마지막에 참기름을 뿌려서 완성합니다.

2. **감자전**
   - **재료**: 감자 2개, 소금 약간, 식용유
   - **레시피**:
     1. 감자를 강판에 갈아서 물기를 제거합니다.
     2. 볼에 갈은 감자와 소금을 섞습니다.
     3. 팬에 식용유를 두르고, 감자 반죽을 올려서 중불에서 양면이 바삭하게 익을 때까지 부칩니다.

3. **감자튀김**
   - **재료**: 감자 4개, 식용유, 소금
   - **레시피**:
     1. 감자를 채 썰어 찬물에 담가 전분을 제거한 후, 물기를 잘 제거합니다.
     2. 깊은 팬에 식용유를 붓고 180도에서 예열합니다.
     3. 감자를 넣고 황금색이 될 때까지 튀긴 후, 소금을 뿌려서 제공합니다.

4. **감자스프**
   - **재료**: 감자 2개, 양파 1개, 마늘 1쪽, 치킨 육수 4컵, 우유 1컵, 소금, 후추
   - **레시피**:
     1. 양파와 마늘을 다지고 감자는 큼직하게 썰어둡니다.
     2. 냄비에 양파와 마늘을 볶다가 감자를 추가하고 치킨 육수를 부어 끓입니다.
     3. 감자가 부드러워지면 블렌더로 갈아서 우유를 추가하고, 소금과 후추로 간을 맞춥니다.

5. **감자 카레**
   - **재료**: 감자 3개, 당근 1개, 양파 1개, 카레 가루 3큰술, 물 3컵
   - **